In [1]:
import altair as alt
from vega_datasets import data
import pandas as pd
import geopandas as gpd

In [2]:
pd.set_option('display.max_columns', None)
df = pd.read_excel("dataset.xlsx", header=5, usecols="A, D:AA",
                   skiprows=[6,7,46,50], na_values='..')
# remove 2 whitespace chars at beginning of each column
df.columns = df.columns.str[2:]
# rename the first header to 'Country'
df.rename(columns={'named: 0': 'Country'}, inplace=True)
# correct the naming scheme of the non-OECD countries
df.iloc[[38], [0]] = 'Brazil'
df.iloc[[39], [0]] = 'Russia'
df.iloc[[40], [0]] = 'South Africa'
# correcting country names to conform to UN standard
df.iloc[[37], [0]] = 'United States of America'
df.iloc[[35], [0]] = 'Turkey'
df.iloc[[30], [0]] = 'Slovakia'
df.iloc[[20], [0]] = 'South Korea'
df.iloc[[7], [0]] = 'Czechia'
print(df['Country'])

0                    Australia
1                      Austria
2                      Belgium
3                       Canada
4                        Chile
5                     Colombia
6                   Costa Rica
7                      Czechia
8                      Denmark
9                      Estonia
10                     Finland
11                      France
12                     Germany
13                      Greece
14                     Hungary
15                     Iceland
16                     Ireland
17                      Israel
18                       Italy
19                       Japan
20                 South Korea
21                      Latvia
22                   Lithuania
23                  Luxembourg
24                      Mexico
25                 Netherlands
26                 New Zealand
27                      Norway
28                      Poland
29                    Portugal
30                    Slovakia
31                    Slovenia
32      

In [3]:
nulls = df.isna().sum()

# convert the binary dataframe into viewing dataframe and create bar chart
visualise_nulls = pd.DataFrame({'Columns': nulls.index, 'Count': nulls.values})

bar = alt.Chart(visualise_nulls).mark_bar().encode(
    x=alt.X('Count:Q'),
    y=alt.Y("Columns:N"),
    text='Count'
)
# add text to label the values of each bar
text = bar.mark_text(
    align='left',
    baseline='middle',
    dx=3
).encode(
    text='Count:Q'
)
bar + text

alt.LayerChart(...)

In [4]:
for column in df.columns:
  if column != 'Country':
    mean_value = df[column].mean()
    df[column].fillna(value=mean_value, inplace=True)

# testing imputations have been implemented correctly
#earnings_mean = df['Personal earnings'].mean()
#print(earnings_mean)
#df[df['Personal earnings'] == earnings_mean]
print(df.columns)
print(df['Life expectancy'].isnull().sum())

Index(['Country', 'Dwellings without basic facilities', 'Housing expenditure',
       'Rooms per person', 'Household net adjusted disposable income',
       'Household net wealth', 'Labour market insecurity', 'Employment rate',
       'Long-term unemployment rate', 'Personal earnings',
       'Quality of support network', 'Educational attainment',
       'Student skills', 'Years in education', 'Air pollution',
       'Water quality', 'Stakeholder engagement for developing regulations',
       'Voter turnout', 'Life expectancy', 'Self-reported health',
       'Life satisfaction', 'Feeling safe walking alone at night',
       'Homicide rate', 'Employees working very long hours',
       'Time devoted to leisure and personal care'],
      dtype='object')
0


In [5]:
# Source of the cartography background
url = "https://naciscdn.org/naturalearth/110m/cultural/ne_110m_admin_0_countries.zip"
countries_shape = gpd.read_file(url) # zipped shapefile
countries_shape = countries_shape[['NAME', 'CONTINENT', 'ISO_A3', 'geometry']]

# enable selection of countries on the map
click_countries  = alt.selection_point(fields=['NAME'])
select_arc = alt.selection_point(fields=['NAME'], empty=False)

countries = (
    alt.Chart(countries_shape).mark_geoshape(
        fill='gray', stroke='black', strokeWidth=0.5
    )
)

sphere = alt.sphere()

basemap = (
    alt.layer(
        alt.Chart(sphere).mark_geoshape(fill='white'),
        alt.Chart(alt.graticule()).mark_geoshape(stroke='black')
    )
)

# encoding data to map
chloropleth = (
    alt.Chart(countries_shape, title='Life Satisfaction')
    .mark_geoshape(stroke='black', strokeWidth=0.5)
    .transform_lookup(
        lookup='NAME',
        from_=alt.LookupData(data=df, key='Country', fields=['Life satisfaction'])
    )
    .encode(
        color='Life satisfaction:Q',
        opacity=alt.condition(click_countries, alt.value(1), alt.value(0.3)),
        tooltip=['NAME', 'Life satisfaction:Q']
    )
    .add_params(click_countries, select_arc)
    .interactive()
)

bars = (
    alt.Chart(countries_shape, title='Life Expectancy')
    .mark_bar(clip=True)
    .transform_lookup(
        lookup='NAME',
        from_=alt.LookupData(data=df, key='Country', fields=['Life expectancy', 'Country', 'Life satisfaction'])
    )
    .encode(
        x=alt.X('Country:N', sort='-y'),
        y=alt.Y('Life expectancy:Q', scale=alt.Scale(domain=[60, 90])),
        opacity=alt.condition(click_countries, alt.value(1), alt.value(0.3)),
        color='Life satisfaction:Q',
        tooltip=['NAME', 'Life satisfaction:Q']
    )
    .add_params(click_countries, select_arc)
    .properties(
        height = 300
    )
)

legend = (
    alt.Chart(countries_shape)
    .mark_point()
    .transform_lookup(
        lookup='NAME',
        from_=alt.LookupData(data=df, key='Country', fields=['Country', 'Life satisfaction'])
    )
    .encode(
        y=alt.Y('Country:N'),
        color=alt.Color('Life satisfaction:Q'),
        opacity=alt.condition(click_countries, alt.value(1), alt.value(0.2))
    )
    .add_params(
        click_countries,
        select_arc
    )
    .properties(
        height=350
    )
)

scatter = (alt.Chart(countries_shape, title='Comparing Employment Rate and Saftey')
    .mark_point()
    #.configure_legend(disable=True)
    .transform_lookup(
        lookup='NAME',
        from_=alt.LookupData(data=df, key='Country', fields=['Employment rate', 'Feeling safe walking alone at night', 'Life satisfaction'])
    )
    .encode(
            x=alt.X('Employment rate:Q').scale(zero=False),
            y=alt.Y('Feeling safe walking alone at night:Q').scale(zero=False),
            opacity=alt.condition(click_countries, alt.value(1), alt.value(0.2)),
            color='Life satisfaction:Q',
            tooltip=['NAME', 'Life satisfaction:Q'],
            #stroke=10
    )
    .add_params(
        click_countries,
        select_arc
    )
    .properties(
        height=300,
        width=400
    )
)


# display the environmental visualisations of only the selected countries
water_arc = (
    alt.Chart(countries_shape, title='Water Quality %')
    .mark_arc(
        radius=25,
        radius2=50,
        cornerRadius=13
    )
    .transform_lookup(
        lookup='NAME',
        from_=alt.LookupData(data=df, key='Country', fields=['Water quality', 'Life satisfaction'])
    )
    .transform_filter(
        click_countries
    )
    .encode(
        theta=alt.Theta('Water quality:Q', scale=alt.Scale(domain=[100, 0])),
        opacity=alt.condition(select_arc, alt.value(1), alt.value(0))
    )
    .properties(
        width=200,
        height=200
    )
)

water_arc_text = water_arc.mark_text(size=25).encode(
    text=('Water quality:Q')
)

health_arc = (
    alt.Chart(countries_shape, title='Self-Reported Health %')
    .mark_arc(
        radius=25,
        radius2=50,
        cornerRadius=13
    )
    .transform_lookup(
        lookup='NAME',
        from_=alt.LookupData(data=df, key='Country', fields=['Self-reported health', 'Life satisfaction'])
    )
    .transform_filter(
        click_countries
    )
    .encode(
        theta=alt.Theta('Self-reported health:Q', scale=alt.Scale(domain=[100, 0])),
        opacity=alt.condition(select_arc, alt.value(1), alt.value(0))
    )
    .properties(
        width=200,
        height=200
    )
)

health_arc_text = health_arc.mark_text(size=25).encode(
    text=('Self-reported health:Q')
)

mcv = alt.vconcat(
    alt.hconcat(
        (basemap + countries + chloropleth).project('equalEarth', scale=150, translate=[410, 200]).properties(width=1000, height=400),
        (water_arc + water_arc_text),
        (health_arc + health_arc_text),
        bounds='full',
        spacing=5
    ),
    alt.hconcat(scatter, legend, bars),
    spacing=3
)
mcv.save('ScatterMCV.html')